In [72]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
from fastai import *
from fastai.vision.all import * 
from torch import optim

In [90]:
from exp.nb_05b import *
torch.set_num_threads(2)

In [91]:
def load_data(folder):
    dataList = []
    labelList = []
    for num in range(10):
        data_path = (path/folder/f'{num}').ls().sorted() #getting path
        
        stackedData = torch.stack([tensor(Image.open(o)) for o in data_path]) #Open each image and stack them
        stackedData = stackedData.float()/255.0 #squishing between 0-1
        
        dataList.append(stackedData) #adding to dataList
        labelList.extend([num]*len(data_path))#extending labelList
    
    #Convert so that each image data is in each row
    train_x = torch.cat(dataList).view(-1, 28*28) 
    train_y = tensor(labelList)
    
    return train_x, train_y

In [92]:
path = untar_data(URLs.MNIST)
path

x_train, y_train = load_data("training")
x_valid, y_valid = load_data("testing")

In [93]:
def normalize(x, m, s): return (x-m)/s

def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    
    return normalize(train, m, s), normalize(valid, m, s)

In [94]:
x_train,x_valid = normalize_to(x_train,x_valid)

In [95]:
x_train.mean(),x_train.std()

(tensor(-1.4117e-09), tensor(1.0000))

In [96]:
dls_train = DataLoader(list(zip(x_train, y_train)), 64, shuffle=True, drop_last=True) 
dls_test = DataLoader(list(zip(x_valid, y_valid)), 128, shuffle=False) 

In [146]:
# dls = DataLoaders(dls_train, dls_test)
dls = DataBunch(dls_train, dls_test)

In [113]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)

def flatten(x):      return x.view(x.shape[0], -1)

In [114]:
def mnist_resize(x): return x.view(-1, 1, 28, 28) #1 channel 28*28

In [115]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
              #inp_c,out_c,filter
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14x14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7x7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4x4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2x2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,10)
    )

In [127]:
model = get_cnn_model(dls)
opt = optim.SGD(model.parameters(), lr=0.4)

In [128]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

learn = Learner(model, opt, loss_func, dls)

In [161]:
cbfs = [Recorder, partial(AvgStatsCallback,accuracy), partial(ParamScheduler,'lr', sched_lin(1., 0.2))]
run = Runner(cb_funcs=cbfs)

In [162]:
run.fit(1, learn)

train: [2.758722881370064, tensor(0.1088)]
valid: [2.30245, tensor(0.1135)]


## Lets do it using our own fit()

In [157]:
model2 = get_cnn_model(dls)
opt2 = optim.SGD(model.parameters(), lr=0.4)

In [158]:
dls_train = DataLoader(list(zip(x_train, y_train)), 64, shuffle=True, drop_last=True) 
dls_test = DataLoader(list(zip(x_valid, y_valid)), 128, shuffle=False) 

In [159]:
def accuracy(preds, yb):
    return (torch.argmax(preds, dim=1) == yb).float().mean()

def fit(epochs):
    for i in range(epochs):
        model2.train()
        
        for xb,yb in dls_train:
            pred = model2(xb)
            loss = F.cross_entropy(pred, yb)
            loss.backward()
            
            opt2.step()
            opt2.zero_grad()
            
        
        model2.eval()
        with torch.no_grad():
            loss,acc = 0.0,0.0
            
            for xb,yb in dls_test:
                pred = model2(xb)
                
                loss += F.cross_entropy(pred, yb)
                acc += accuracy(pred, yb)
                
        nv = len(dls_test)
        print(i, loss/nv, acc/nv)

In [160]:
fit(1)

0 tensor(2.3082) tensor(0.0804)
